### Practice Data

I will be using the 'Maji Ndogo' (From ALX Coursework) to practice pandas concepts and record new things I learn particularly in pandas

In [1]:
import pandas as pd 
from sqlalchemy import create_engine, text 

# Create an engine for the database
engine = create_engine('sqlite:///Maji_Ndogo_farm_survey_small.db') 

In [2]:
#Testing connection by printing out the names of all tables
with engine.connect() as connection:
    result = connection.execute(text("SELECT name FROM sqlite_master WHERE type='table';"))
    for row in result:
        print(row)

('geographic_features',)
('weather_features',)
('soil_and_crop_features',)
('farm_management_features',)


In [4]:
#Connecting all tables to make a single Dataframe

sql_query = """
SELECT *
FROM geographic_features g
JOIN weather_features w
    ON g.Field_ID = w.Field_ID
JOIN soil_and_crop_features s
    ON g.Field_ID = s.Field_ID
JOIN farm_management_features f
    ON g.Field_ID = f.Field_ID;
"""

# Create a connection object
with engine.connect() as connection:
    
    # Use Pandas to execute the query and store the result in a DataFrame
    df = pd.read_sql_query(text(sql_query), connection)

In [6]:
# drop all columns named 'Field_ID'
df.drop(columns = 'Field_ID', inplace = True)

In [12]:
# Insert your code here
columns_list = df.columns.tolist() #creating a list of all df columns
df.columns = ['Elevation','Latitude','Longitude',
                       'Location','Slope','Rainfall',
                       'Min_temperature_C','Max_temperature_C',
                       'Ave_temps','Soil_fertility','Soil_type',
                       'pH','Pollution_level','Plot_size',
                       'Annual_yield','Crop_type','Standard_yield'] #swapped the column annual_yield and crop type


#change spelling mistakes
corrections = {
    "cassaval": "cassava",
    "cassava ": "cassava",
    "wheatn": "wheat",
    "wheat ": "wheat",
    "teaa": "tea",
    "tea ": "tea"
}

df['Crop_type'] = df['Crop_type'].replace(corrections)

#changing elevation to positive
df['Elevation'] = df['Elevation'].abs()

In [13]:
df.head()

,Elevation,Latitude,Longitude,Location,Slope,Rainfall,Min_temperature_C,Max_temperature_C,Ave_temps,Soil_fertility,Soil_type,pH,Pollution_level,Plot_size,Annual_yield,Crop_type,Standard_yield
0,786.05580,-7.389911,-7.556202,Rural_Akatsi,14.795113,1125.2,-3.1,33.1,15.00,0.62,Sandy,6.169393,0.085267,1.3,0.751354,cassava,0.577964
1,674.33410,-7.736849,-1.051539,Rural_Sokoto,11.374611,1450.7,-3.9,30.6,13.35,0.64,Volcanic,5.676648,0.399684,2.2,1.069865,cassava,0.486302
2,826.53390,-9.926616,0.115156,Rural_Sokoto,11.339692,2208.9,-1.8,28.4,13.30,0.69,Volcanic,5.331993,0.358029,3.4,2.208801,tea,0.649647
3,574.94617,-2.420131,-6.592215,Rural_Kilimani,7.109855,328.8,-5.8,32.2,13.20,0.54,Loamy,5.328150,0.286687,2.4,1.277635,cassava,0.532348
4,886.35300,-3.055434,-7.952609,Rural_Kilimani,55.007656,785.2,-2.5,31.0,14.25,0.72,Sandy,5.721234,0.043190,1.5,0.832614,wheat,0.555076


### Filtering DataFrames

.loc

- used when filtering based on labels e.g. df.loc[row_label]
- useful when conditional filtering e.g. df.loc[condition]

.iloc

- used to filter based on row indexes e.g. df.iloc[3: ]

In [16]:
#iloc: fourth to 10th row

df.iloc[3:10]

,Elevation,Latitude,Longitude,Location,Slope,Rainfall,Min_temperature_C,Max_temperature_C,Ave_temps,Soil_fertility,Soil_type,pH,Pollution_level,Plot_size,Annual_yield,Crop_type,Standard_yield
3,574.94617,-2.420131,-6.592215,Rural_Kilimani,7.109855,328.8,-5.8,32.2,13.20,0.54,Loamy,5.328150,2.866871e-01,2.4,1.277635,cassava,0.532348
4,886.35300,-3.055434,-7.952609,Rural_Kilimani,55.007656,785.2,-2.5,31.0,14.25,0.72,Sandy,5.721234,4.319027e-02,1.5,0.832614,wheat,0.555076
5,850.56647,-2.050665,-7.132769,Rural_Kilimani,50.451250,649.4,-3.0,29.5,13.25,0.70,Sandy,5.484350,1.275280e-01,1.7,1.112611,potato,0.654477
6,331.35538,-13.409517,-6.722849,Rural_Hawassa,5.907423,1586.0,-6.8,31.8,12.50,0.63,Silt,7.061317,4.315457e-07,2.9,0.994325,banana,0.342871
7,790.73846,-3.785356,-1.470468,Rural_Kilimani,15.380596,1188.3,-3.0,31.0,14.00,0.63,Sandy,4.577043,6.131074e-01,1.4,0.558899,tea,0.399214
8,742.72180,-10.237509,-0.736216,Rural_Sokoto,3.431843,2143.3,-2.6,28.2,12.80,0.66,Volcanic,5.447321,2.784792e-01,13.9,8.532665,coffee,0.613861
9,747.38030,-3.767334,-3.105643,Rural_Kilimani,8.239554,796.6,-3.8,32.7,14.45,0.58,Sandy,4.897414,3.103862e-01,4.6,2.214679,cassava,0.481452


In [15]:
def find_good_conditions(df, crop_type):
    # filter conditions
    row_filter0 = (df['Crop_type'] == crop_type)
    row_filter1 = (df['Standard_yield'] > df['Standard_yield'].mean())
    row_filter2 = ((df['Ave_temps'] >=12) & (df['Ave_temps'] <= 15))
    row_filter3 = (df['Pollution_level'] < 0.0001)
    
    #How to combine conditions
    combined_condition = row_filter0 & row_filter1 & row_filter2 & row_filter3
    
    return df.loc[combined_condition]

find_good_conditions(df, "tea")

,Elevation,Latitude,Longitude,Location,Slope,Rainfall,Min_temperature_C,Max_temperature_C,Ave_temps,Soil_fertility,Soil_type,pH,Pollution_level,Plot_size,Annual_yield,Crop_type,Standard_yield
197,688.63477,-14.585503,-5.948055,Rural_Hawassa,2.506081,1569.9,-3.6,31.6,14.00,0.62,Volcanic,4.117729,1.007224e-08,10.4,7.484284,tea,0.719643
852,739.42730,-14.405275,-6.267883,Rural_Hawassa,7.132496,1687.9,-3.1,31.1,14.00,0.64,Volcanic,4.538485,7.108529e-09,2.1,1.664479,tea,0.792609
869,691.24520,-14.400770,-5.966074,Rural_Hawassa,11.563782,1640.5,-3.5,32.7,14.60,0.65,Volcanic,4.123869,1.691890e-08,3.0,2.219570,tea,0.739857
943,713.74615,-14.549458,-6.051661,Rural_Hawassa,14.432365,1604.0,-3.4,27.7,12.15,0.66,Volcanic,5.147911,7.575035e-09,0.7,0.554950,tea,0.792785
1293,648.13605,-14.297139,-6.146258,Rural_Hawassa,9.684363,1648.5,-3.8,29.5,12.85,0.65,Volcanic,5.154659,1.387864e-08,2.5,2.095446,tea,0.838179
1376,660.65173,-14.423298,-6.011120,Rural_Hawassa,7.688627,1603.4,-3.8,28.8,12.50,0.64,Volcanic,4.330087,1.364089e-08,3.4,2.648997,tea,0.779117
2010,527.61390,-14.161968,-6.731858,Rural_Hawassa,0.977717,1560.2,-5.1,33.3,14.10,0.61,Peaty,5.092398,1.150044e-08,9.3,6.679613,tea,0.718238
2278,667.37920,-14.382747,-6.632757,Rural_Hawassa,2.563133,1625.9,-3.8,31.4,13.80,0.62,Volcanic,4.844913,4.868797e-09,14.3,11.751984,tea,0.821817
3445,678.59955,-14.648583,-6.083194,Rural_Hawassa,16.919823,1531.3,-3.8,28.4,12.30,0.66,Volcanic,4.828436,4.663686e-09,2.0,1.580439,tea,0.790220
3568,684.76980,-14.261093,-6.267883,Rural_Hawassa,21.928755,1689.2,-3.5,29.2,12.85,0.69,Volcanic,4.815771,1.248680e-08,0.6,0.499929,tea,0.833216


### Conditional Filtering

In [17]:
#using df.query to use sql-like filters on a dataframe

df.query('Standard_yield > 0.5 and Soil_type == "Loamy"')

,Elevation,Latitude,Longitude,Location,Slope,Rainfall,Min_temperature_C,Max_temperature_C,Ave_temps,Soil_fertility,Soil_type,pH,Pollution_level,Plot_size,Annual_yield,Crop_type,Standard_yield
3,574.94617,-2.420131,-6.592215,Rural_Kilimani,7.109855,328.8,-5.8,32.2,13.20,0.54,Loamy,5.328150,0.286687,2.4,1.277635,cassava,0.532348
19,610.99400,-6.948353,-2.966000,Rural_Hawassa,16.369598,902.8,-4.9,28.4,11.75,0.61,Loamy,4.698949,0.303487,1.5,0.791114,wheat,0.527409
22,504.42505,-2.379580,-7.601249,Rural_Kilimani,3.906222,335.3,-6.4,32.6,13.10,0.53,Loamy,5.429212,0.057291,7.1,4.256354,wheat,0.599486
26,525.38104,-2.465188,-5.186775,Rural_Kilimani,6.809244,298.6,-6.3,33.8,13.75,0.54,Loamy,6.453105,0.192563,4.8,2.603143,potato,0.542321
44,562.14720,-7.953123,-2.222739,Rural_Sokoto,24.619905,1200.3,-5.3,31.8,13.25,0.66,Loamy,4.649274,0.262477,1.4,0.948407,tea,0.677433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5617,636.75790,-7.322326,-5.128215,Rural_Hawassa,17.667543,812.0,-4.8,30.6,12.90,0.61,Loamy,4.882120,0.193366,0.8,0.442964,cassava,0.553705
5621,509.44156,-6.614932,-1.641644,Rural_Sokoto,19.441830,958.9,-5.8,31.5,12.85,0.62,Loamy,5.485380,0.800948,1.7,0.943078,cassava,0.554751
5625,614.65955,-6.948353,-3.029065,Rural_Hawassa,2.801264,896.6,-4.9,28.5,11.80,0.57,Loamy,4.809189,0.288357,5.3,3.053344,potato,0.576103
5632,618.01624,-6.511301,-2.574099,Rural_Hawassa,19.183754,888.0,-4.9,26.9,11.00,0.62,Loamy,5.231630,0.499646,0.7,0.350604,wheat,0.500862
